In [15]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

In [16]:
stocks = pd.read_excel (r'C:\Users\65883\Documents\SnP500.xlsx')

In [17]:
IEX_CLOUD_API_TOKEN = 'pk_347c02cc98504f2d852a657ba3ac9ab0'
IEX_SANDBOX_API_TOKEN = 'Tsk_bb6e306f3c4449b298d3bbfc08422dc1'

In [18]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [19]:
symbol_groups = list(chunks(stocks['Stock'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

In [20]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio')
        
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio1000000
1000000


In [21]:
rv_columns = [
    'Ticker',
    'Company Name',
    'Price',
    'Number of Shares to buy',
    'PE Ratio',
    'PE Percentile',
    'PB Ratio',
    'PB Percentile',
    'PS Ratio',
    'PS Percentile',
    'EV to EBITDA Ratio',
    'EV/EBITDA Percentile',
    'EV to GrossProfit Ratio',
    'EV/GrossProfit Percentile',
    'Value Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)
rv_dataframe

,Ticker,Company Name,Price,Number of Shares to buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV to EBITDA Ratio,EV/EBITDA Percentile,EV to GrossProfit Ratio,EV/GrossProfit Percentile,Value Score


In [22]:
hqm_columns = [
    'Ticker',
    'Company Name',
    'Price',
    'Number of Shares to buy',
    'One year price return',
    'One year return percentile',
    'Six month price return',
    'Six month return percentile',
    'Three month price return',
    'Three month return percentile',
    'One month price return',
    'One month return percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)
hqm_dataframe

,Ticker,Company Name,Price,Number of Shares to buy,One year price return,One year return percentile,Six month price return,Six month return percentile,Three month price return,Three month return percentile,One month price return,One month return percentile,HQM Score


In [23]:
for symbol_string in symbol_strings:
    print(symbol_string)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,stats,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    #batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,stats,advanced-stats&token={IEX_SANDBOX_API_TOKEN}'
    requests.get(batch_api_call_url).json()
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue'] 
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit'] 
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        rv_dataframe = rv_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['companyName'],
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
            ],
            index = rv_columns),
            ignore_index = True
        )
        
        
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['stats']['companyName'],
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ],
            index = hqm_columns),
            ignore_index = True
        )
        


AOS,ABT,ABBV,ABMD,ACN,ATVI,ADM,ADBE,AAP,AMD,AES,AFL,A,APD,AKAM,ALK,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,ABC,AME,AMGN,APH,ADI,ANSS,ANTM,AON,APA,AAPL,AMAT,APTV,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,BKR,BLL,BAC,BBWI,BAX,BDX,BRK.B,BBY,BIO,TECH,BIIB,BLK,BK,BA,BKNG,BWA,BXP,BSX,BMY,AVGO,BR,BRO,BF.B,CHRW,CDNS,CZR,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,CNC,CNP,CDAY,CERN,CF


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
rv_dataframe

In [ ]:
hqm_dataframe

In [ ]:
for column in ['PE Ratio', 'PB Ratio', 'PS Ratio', 'EV to EBITDA Ratio', 'EV to GrossProfit Ratio']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [ ]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

In [ ]:
metrics ={'PE Ratio': 'PE Percentile',
    'PB Ratio': 'PB Percentile',
    'PS Ratio': 'PS Percentile',
    'EV to EBITDA Ratio': 'EV/EBITDA Percentile',
    'EV to GrossProfit Ratio': 'EV/GrossProfit Percentile',
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])

        
rv_dataframe

In [ ]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'Value Score'] = mean(value_percentiles)

rv_dataframe

In [ ]:
rv_dataframe.sort_values('Value Score', ascending = True, inplace = True)
rv_dataframe =rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

In [ ]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to buy'] = math.floor(position_size / rv_dataframe.loc[row, 'Price'])
    
rv_dataframe

In [ ]:
time_periods = [
                'One year',
                'Six month',
                'Three month',
                'One month'
              ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} return percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} price return'], hqm_dataframe.loc[row, f'{time_period} price return'])
        
hqm_dataframe

In [ ]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} return percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

In [ ]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

In [ ]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for row in hqm_dataframe.index:
    hqm_dataframe.loc[row, 'Number of Shares to buy'] = math.floor(position_size / hqm_dataframe.loc[row, 'Price'])
    
hqm_dataframe

In [ ]:
writer = pd.ExcelWriter('SnP500Momentum_Portfolio.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, 'SnP500Momentum_Portfolio', index =  False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
        
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
        
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
        
    }
)    
decimal_format = writer.book.add_format(
    {
        'num_format': '0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
        
    }
)

In [ ]:
writer.sheets['SnP500Momentum_Portfolio'].write('A1', 'Ticker', string_format)
writer.sheets['SnP500Momentum_Portfolio'].write('B1', 'Company Name', string_format)
writer.sheets['SnP500Momentum_Portfolio'].write('C1', 'Price', dollar_format)
writer.sheets['SnP500Momentum_Portfolio'].write('D1', 'Number of Shares to buy', integer_format)
writer.sheets['SnP500Momentum_Portfolio'].write('E1', 'One year price return', decimal_format)
writer.sheets['SnP500Momentum_Portfolio'].write('F1', 'One year return percentile', decimal_format)
writer.sheets['SnP500Momentum_Portfolio'].write('G1', 'Six month price return', decimal_format)
writer.sheets['SnP500Momentum_Portfolio'].write('H1', 'Six month return percentile', decimal_format)
writer.sheets['SnP500Momentum_Portfolio'].write('I1', 'Three month price return', decimal_format)
writer.sheets['SnP500Momentum_Portfolio'].write('J1', 'Three month return percentile', decimal_format)
writer.sheets['SnP500Momentum_Portfolio'].write('K1', 'One month price return', decimal_format)
writer.sheets['SnP500Momentum_Portfolio'].write('L1', 'One month return percentile', decimal_format)
writer.sheets['SnP500Momentum_Portfolio'].write('M1', 'HQM Score', decimal_format)


In [ ]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Company Name', string_format],
    'C': ['Price', dollar_format],
    'D': ['Number of Shares to buy', integer_format],
    'E': ['One year price return', decimal_format],
    'F': ['One year return percentile', decimal_format],
    'G': ['Six month price return', decimal_format],
    'H': ['Six month return percentile', decimal_format],
    'I': ['Three month price return', decimal_format],
    'J': ['Three month return percentile', decimal_format],
    'K': ['One month price return', decimal_format],
    'L': ['One month return percentile', decimal_format],
    'M': ['HQM Score', decimal_format]
}

for column in column_formats.keys():
    writer.sheets['SnP500Momentum_Portfolio'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['SnP500Momentum_Portfolio'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    
writer.save()

In [ ]:
writer = pd.ExcelWriter('SnP500Value_Portfolio.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, 'SnP500Value_Portfolio', index =  False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
        
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
        
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
        
    }
)    
decimal_format = writer.book.add_format(
    {
        'num_format': '0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
        
    }
)

In [ ]:
writer.sheets['SnP500Value_Portfolio'].write('A1', 'Ticker', string_format)
writer.sheets['SnP500Value_Portfolio'].write('B1', 'Company Name', string_format)
writer.sheets['SnP500Value_Portfolio'].write('C1', 'Price', dollar_format)
writer.sheets['SnP500Value_Portfolio'].write('D1', 'Number of Shares to buy', integer_format)
writer.sheets['SnP500Value_Portfolio'].write('E1', 'PE Ratio', decimal_format)
writer.sheets['SnP500Value_Portfolio'].write('F1', 'PE Percentile', decimal_format)
writer.sheets['SnP500Value_Portfolio'].write('G1', 'PB Ratio', decimal_format)
writer.sheets['SnP500Value_Portfolio'].write('H1', 'PB Percentile', decimal_format)
writer.sheets['SnP500Value_Portfolio'].write('I1', 'PS Ratio', decimal_format)
writer.sheets['SnP500Value_Portfolio'].write('J1', 'PS Percentile', decimal_format)
writer.sheets['SnP500Value_Portfolio'].write('K1', 'EV to EBITDA Ratio', decimal_format)
writer.sheets['SnP500Value_Portfolio'].write('L1', 'EV/EBITDA Percentile', decimal_format)
writer.sheets['SnP500Value_Portfolio'].write('M1', 'EV to GrossProfit Ratio', decimal_format)
writer.sheets['SnP500Value_Portfolio'].write('N1', 'EV/GrossProfit Percentile', decimal_format)
writer.sheets['SnP500Value_Portfolio'].write('O1', 'Value Score', decimal_format)

In [ ]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Company Name', string_format],
    'C': ['Price', dollar_format],
    'D': ['Number of Shares to buy', integer_format],
    'E': ['PE Ratio', decimal_format],
    'F': ['PE Percentile', decimal_format],
    'G': ['PB Ratio', decimal_format],
    'H': ['PB Percentile', decimal_format],
    'I': ['PS Ratio', decimal_format],
    'J': ['PS Percentile', decimal_format],
    'K': ['EV to EBITDA Ratio', decimal_format],
    'L': ['EV/EBITDA Percentile', decimal_format],
    'M': ['EV to GrossProfit Ratio', decimal_format],
    'N': ['EV/GrossProfit Percentile', decimal_format],
    'O': ['Value Score', decimal_format],
}

for column in column_formats.keys():
    writer.sheets['SnP500Value_Portfolio'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['SnP500Value_Portfolio'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    
writer.save()